In [27]:
# start by defining the codespace using qutip
# |g+f, g+f>; |g-f, g-f> are the basis states
from qutip import *
import numpy as np

t_levels = 3

p_gf = (basis(t_levels, 0) + basis(t_levels, 2)).unit()
m_gf = (basis(t_levels, 0) - basis(t_levels, 2)).unit()

# tensor 2 qubits together to get logical qubit
logical0 = tensor(p_gf, p_gf)
logical1 = tensor(m_gf, m_gf)

# verify that the basis states are orthogonal
assert not logical0.overlap(logical1)

In [28]:
# add an ancilla flag qubit in the |g+e> state
p_gf = (basis(t_levels, 0) + basis(t_levels, 2)).unit()
m_gf = (basis(t_levels, 0) - basis(t_levels, 2)).unit()
# p_ge = (basis(t_levels, 0) + basis(t_levels, 1)).unit()
p_g = basis(2, 0).unit()
p_e = basis(2, 1).unit()
# m_ge = (basis(t_levels, 0) - basis(t_levels, 1)).unit()

# tensor 2 qubits together to get logical qubit
# keep the flag |g+e> for no error
logical0 = tensor(p_gf, p_gf, p_g)
logical1 = tensor(m_gf, m_gf, p_g)

error0 = tensor(p_gf, p_gf, p_e)
error1 = tensor(m_gf, m_gf, p_e)

# verify that the basis states are orthogonal
assert not logical0.overlap(logical1)
assert not error0.overlap(error1)
assert not logical0.overlap(error0)
assert not logical0.overlap(error1)

In [29]:
# next, define the stabilizers for the code
stabilizers = []
# X^02_1 X^01_2
# define operator Xgf, which is |g><f| + |f><g|
Xgf = tensor(
    basis(t_levels, 0) * basis(t_levels, 2).dag()
    + basis(t_levels, 2) * basis(t_levels, 0).dag()
)
Xge = tensor(basis(2, 0) * basis(2, 1).dag() + basis(2, 1) * basis(2, 0).dag())

# XgfXgf = tensor(Xgf, Xgf, qeye(2))
# stabilizers.append(XgfXgf)

# project into |0><0|_L + |1><1|_L
logical_projector = logical0 * logical0.dag() + logical1 * logical1.dag()
stabilizers.append(logical_projector)

# # P1_1, P1_2 are projection operators onto |e> on qubit 1 and 2
# # I think...
# P1_1 = tensor(basis(t_levels, 1) * basis(t_levels, 1).dag(), qeye(t_levels))
# P1_2 = tensor(qeye(t_levels), basis(t_levels, 1) * basis(t_levels, 1).dag())
# stabilizers.append(P1_1)
# stabilizers.append(P1_2)

# verify that the stabilizers all stabilize the basis states
for s in stabilizers:
    assert s * logical0 == logical0
    assert s * logical1 == logical1

In [30]:
# next define the logical operators, which are defined as centralizers of the stabilizers
# logical_operators = [IX, XI, XX, YY, ZZ, YZ, ZY]
logical_operators = []

# Zgf = |g><g| - |f><f|
Zgf = tensor(
    basis(t_levels, 0) * basis(t_levels, 0).dag()
    - basis(t_levels, 2) * basis(t_levels, 2).dag()
)
ZgfZgf = tensor(Zgf, Zgf, qeye(2))
logical_operators.append(ZgfZgf)

IXgf = tensor(qeye(t_levels), Xgf, qeye(2))
XgfI = tensor(Xgf, qeye(t_levels), qeye(2))
logical_operators.append(IXgf)
logical_operators.append(XgfI)

# Ygf = -1j|g><f| + 1j|f><g|
Ygf = tensor(
    -1j * basis(t_levels, 0) * basis(t_levels, 2).dag()
    + 1j * basis(t_levels, 2) * basis(t_levels, 0).dag()
)
YgfYgf = tensor(Ygf, Ygf, qeye(2))
logical_operators.append(YgfYgf)

YgfZgf = tensor(Ygf, Zgf, qeye(2))
ZgfYgf = tensor(Zgf, Ygf, qeye(2))
logical_operators.append(YgfZgf)
logical_operators.append(ZgfYgf)

In [31]:
# verify that the logical operators commute with the stabilizers
# def is_centralizer(s, l):
#     for s in stabilizers:
#         if not all(s * l == l * s):
#             return False
#     return True
def is_centralizer(l):
    if not all(s * l == l * s for s in stabilizers):
        return False
    return True


for l in logical_operators:
    print(is_centralizer(l))

True
True
True
True
True
True


In [32]:
# for each logical operator, need to define its behavior on the logical basis states
# for eaxmple,
# IX|0>_L = IX|g+f, g+f> = |g+f, g+f> = |0>_L
# IX|1>_L = IX|g-f, g-f> = -|g-f, g-f> = -|1>_L
# so IX acts as Z_L on the logical basis states


# now do this programmatically
# IXgf  * logical1 = a * logical0 + b * logical1
# solve for a and b
def convert_to_logical_op(op):
    logical_op = []
    for basis in [logical0, logical1]:
        a = (op * basis).overlap(logical0)
        b = (op * basis).overlap(logical1)
        logical_op.append((a, b))
    logical_op = np.array(logical_op).T
    return logical_op


# check operator with respect to flag qubit
def convert_to_full_op(op):
    logical_op = []
    for basis in [logical0, logical1, error0, error1]:
        a = (op * basis).overlap(logical0)
        b = (op * basis).overlap(logical1)
        c = (op * basis).overlap(error0)
        d = (op * basis).overlap(error1)
        logical_op.append((a, b, c, d))
    logical_op = np.array(logical_op).T
    return logical_op

In [33]:
logical_op = convert_to_logical_op(IXgf)
print(logical_op)  # should be logical Z
assert np.allclose(logical_op, sigmaz())

[[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]


In [34]:
logical_op = convert_to_logical_op(ZgfZgf)
print(logical_op)  # should be logical X
assert np.allclose(logical_op, sigmax())

[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]


In [35]:
logical_op = convert_to_logical_op(YgfZgf)
print(logical_op)  # should be logical Y
assert np.allclose(logical_op, sigmay())

[[0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j]]


In [11]:
# print(a.dag())

In [36]:
# next, build the operators directly from parametric drive Hamiltonian
# define raising and lowering operators
a = tensor(destroy(t_levels), qeye(t_levels), qeye(2))
b = tensor(qeye(t_levels), destroy(t_levels), qeye(2))
c = tensor(destroy(t_levels), destroy(t_levels), qeye(2))

# define the parametric drive Hamiltonian as the coefficents between each of the possible states
# H = xa + xb + xc + xab + xac + xbc + xabc
# H += xa^dag + xb^dag + xc^dag +  xa^dag b + xab^dag + xa^dag b^dag + xa^dag c + xac^dag + xa^dag c^dag + ...
# H += xaa + xbb + xcc + xaa^dag + xbb^dag + xcc^dag + ...

# want to build a Hamiltonian -> write as unitary operator
# if operator centralizes stabilizers, then it is a logical operator
# then output the operator in the logical basis representation

# example: (b^dag b^dag + bb)(c^dag + c) => XgfXe => I_L
H = (b.dag() * b.dag() + b * b) * (c.dag() + c)
U = (-1j * H).expm()
logical_op = convert_to_logical_op(U)
print(logical_op)

# NOTE not a centralizer
print(is_centralizer(U))

[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
False


In [13]:
# XXX is this wrong?
detect_gate = tensor(Xgf, Xgf, Xge)
# H = ((a.dag()*a.dag() + a*a)+(b.dag()*b.dag() + b*b))*(c.dag() + c)
# U = (-1j*H).expm()

# # XXX looks to be broken ...
# # should take raise flag if phase flip error occurs
# # full_op = convert_to_full_op(U)
# # print(full_op)

# error0 = |g+f, g-f, g>
flag_qubit = error0.ptrace(2)
print(flag_qubit)

flag_state = detect_gate * error0
flag_qubit = flag_state.ptrace(2)
print(flag_qubit)

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 0.]
 [0. 1.]]
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]


In [14]:
# print(a + a.dag())
H = (a + a.dag()).full()
H = H.real

H = logical_projector.full().real
# set numpy print options
np.set_printoptions(precision=3, suppress=False, linewidth=400)
print(np.array2string(H[::2, ::2], separator=", "))

[[0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0.5, 0. , 0. , 0. , 0.5, 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0. , 0. , 0.5, 0. , 0. , 0. , 0.5, 0. , 0. ],
 [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ],
 [0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0.5]]


In [15]:
import itertools

# Map the strings to the actual operators
parametric_drives_op = {
    "a": a,
    "b": b,
    "c": c,
    "a†": a.dag(),
    "b†": b.dag(),
    "c†": c.dag(),
}

# Create the string list from the keys of the dictionary
parametric_drives_str = list(parametric_drives_op.keys())

# Define the maximum number of terms in the Hamiltonian
max_terms = 3

# Define the maximum number of terms in the sum
max_sum_terms = 2

# Define the maximum power of each term (you can change this value as needed)
max_power = 2

# Iterate over all combinations of the parametric drives
for r in range(1, max_terms + 1):
    for combination in itertools.product(parametric_drives_str, repeat=r):
        # Create the Hamiltonian as the product of the terms
        H = parametric_drives_op[combination[0]]
        for i in range(1, r):
            H = H * parametric_drives_op[combination[i]]

        # Now add combinations with powers up to max_power
        for p in range(2, max_power + 1):
            H_power = H**p

            # Now consider sums of the Hamiltonian up to max_sum_terms
            for s in range(1, max_sum_terms + 1):
                for sum_combination in itertools.combinations([H_power] * s, s):
                    H_sum = sum(sum_combination)

                    # add hermitian conjugate
                    H_sum += H_sum.dag()

                    # Generate the unitary operator from the Hamiltonian
                    U = (-1j * H_sum).expm()

                    # Check if the unitary operator is a logical operator
                    logical_op = convert_to_logical_op(U)

                    # Add some condition to decide if it's a "logical operator"
                    if is_centralizer(H_sum) and not np.allclose(logical_op, np.eye(2)):
                        print(
                            "Found logical operator: ",
                            " + ".join(["".join(combination)] * s),
                            " + h.c.",
                        )

                        # Print the logical operator in the logical basis representation
                        print(logical_op)
                        print("\n")

Found logical operator:  a  + h.c.
[[0.156+0.988j 0.   +0.j   ]
 [0.   +0.j    0.156-0.988j]]


Found logical operator:  a + a  + h.c.
[[-0.951+0.308j  0.   +0.j   ]
 [ 0.   +0.j    -0.951-0.308j]]


Found logical operator:  b  + h.c.
[[0.156+0.988j 0.   +0.j   ]
 [0.   +0.j    0.156-0.988j]]


Found logical operator:  b + b  + h.c.
[[-0.951+0.308j  0.   +0.j   ]
 [ 0.   +0.j    -0.951-0.308j]]


Found logical operator:  c  + h.c.
[[ 0.292+0.455j -0.708+0.455j]
 [-0.708+0.455j  0.292+0.455j]]


Found logical operator:  c + c  + h.c.
[[ 0.173-0.378j -0.827-0.378j]
 [-0.827-0.378j  0.173-0.378j]]


Found logical operator:  a†  + h.c.
[[0.156+0.988j 0.   +0.j   ]
 [0.   +0.j    0.156-0.988j]]


Found logical operator:  a† + a†  + h.c.
[[-0.951+0.308j  0.   +0.j   ]
 [ 0.   +0.j    -0.951-0.308j]]


Found logical operator:  b†  + h.c.
[[0.156+0.988j 0.   +0.j   ]
 [0.   +0.j    0.156-0.988j]]


Found logical operator:  b† + b†  + h.c.
[[-0.951+0.308j  0.   +0.j   ]
 [ 0.   +0.j    -0.951-0